# Eigenvalues of the dielectric matrix

We compute a few eigenvalues of the dielectric matrix ($q=0$, $ω=0$) iteratively.

In [1]:
using DFTK
using Plots
using KrylovKit
using Printf

# Calculation parameters
kgrid = [1, 1, 1]
Ecut = 5

# Silicon lattice
a = 10.26
lattice = a / 2 .* [[0 1 1.]; [1 0 1.]; [1 1 0.]]
Si = ElementPsp(:Si, psp=load_psp("hgh/lda/Si-q4"))
atoms     = [Si, Si]
positions = [ones(3)/8, -ones(3)/8]

# Compute the dielectric operator without symmetries
model = model_LDA(lattice, atoms, positions, symmetries=false)
basis = PlaneWaveBasis(model; Ecut, kgrid)
scfres = self_consistent_field(basis, tol=1e-14);

n     Energy            log10(ΔE)   log10(Δρ)   Diag
---   ---------------   ---------   ---------   ----
  1   -7.234444511576                   -0.50    7.0
  2   -7.249877160990       -1.81       -1.39    1.0
  3   -7.250954752467       -2.97       -1.81    4.0
  4   -7.250933621382   +   -4.68       -1.86    2.0
  5   -7.251325789123       -3.41       -2.59    2.0
  6   -7.251338319265       -4.90       -3.21    2.0
  7   -7.251338771105       -6.35       -3.79    2.0
  8   -7.251338794369       -7.63       -4.17    3.0
  9   -7.251338798563       -8.38       -4.97    1.0
 10   -7.251338798690       -9.90       -5.41    3.0
 11   -7.251338798704      -10.84       -6.32    3.0
 12   -7.251338798704      -12.75       -6.58    3.0
 13   -7.251338798705      -13.05       -7.31    3.0
 14   -7.251338798705      -14.21       -8.04    3.0


Applying $ε^† ≔ (1- χ_0 K)$ …

In [2]:
function eps_fun(δρ)
    δV = apply_kernel(basis, δρ; ρ=scfres.ρ)
    χ0δV = apply_χ0(scfres, δV)
    δρ - χ0δV
end;

… eagerly diagonalizes the subspace matrix at each iteration

In [3]:
eigsolve(eps_fun, randn(size(scfres.ρ)), 5, :LM; eager=true, verbosity=3);

[ Info: Arnoldi iteration step 1: normres = 0.05254221317253042
[ Info: Arnoldi iteration step 2: normres = 0.6501675652734968
[ Info: Arnoldi iteration step 3: normres = 0.7736343549555188
[ Info: Arnoldi iteration step 4: normres = 0.3334576396202904
[ Info: Arnoldi iteration step 5: normres = 0.3868593512293453
[ Info: Arnoldi schursolve in iter 1, krylovdim = 5: 0 values converged, normres = (2.39e-02, 6.90e-02, 3.07e-01, 2.22e-01, 2.50e-02)
[ Info: Arnoldi iteration step 6: normres = 0.291837447052941
[ Info: Arnoldi schursolve in iter 1, krylovdim = 6: 0 values converged, normres = (6.48e-03, 1.85e-01, 1.98e-01, 8.05e-02, 1.91e-02)
[ Info: Arnoldi iteration step 7: normres = 0.08710750871220727
[ Info: Arnoldi schursolve in iter 1, krylovdim = 7: 0 values converged, normres = (2.68e-04, 1.54e-02, 1.09e-02, 2.65e-02, 7.61e-02)
[ Info: Arnoldi iteration step 8: normres = 0.10732984648661881
[ Info: Arnoldi schursolve in iter 1, krylovdim = 8: 0 values converged, normres = (1.22e-05